In [18]:
import csv
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import *
import math
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [30]:
canto_data = pd.read_csv('./data/full_cantometrics.csv').drop(columns=['Unnamed: 0'])

In [36]:
# canto_data = pd.read_csv('./output/full_cantometrics.csv')
features = ['line_'+str(i+1)for i in range(37)]
cultures = canto_data['Preferred_name'].unique()
columns = ['Preferred_name', 'soc_id']
for feature in features:
    columns.append(feature)

In [38]:
%%notify -m "final matrix imputed"
final_df = pd.DataFrame(columns = columns)
for culture in cultures:
    modal_profile = dict(canto_data[canto_data['Preferred_name']==culture][features].mode().loc[0])
    modal_profile['Preferred_name'] = culture
    soc_id = canto_data[canto_data['Preferred_name']==culture]['society_id'].reset_index()['society_id'][0]
    modal_profile['soc_id'] = soc_id
    final_df = final_df.append(modal_profile, ignore_index=True)

<IPython.core.display.Javascript object>

In [41]:
metadata = pd.read_csv('./data/metadata.csv')

In [64]:
for i, row in final_df.iterrows():
    culture = row['Preferred_name']
    try:
        lat = metadata[metadata['Preferred_name']==culture].reset_index()['Local_latitude'][0]
        lng = metadata[metadata['Preferred_name']==culture].reset_index()['Local_longitude'][0]
    except:
        final_df.drop([i])
    final_df.loc[i, ['lat']] = lat
    final_df.loc[i, ['lng']] = lng

In [66]:
final_df.to_csv('././output/modal_profiles.csv')

## Modal Profiles without multi-codings

In [110]:
downloaded_data = pd.read_csv('./data/downloaded_data.csv')

In [111]:
# Load conversion guide
with open('./data/conversion_guide.json') as f:
  conversion_guide = json.load(f)

df = downloaded_data.drop(columns=['orv_1', 'orv_2', 'ensemble_value_id',
       'ensemble_value_label', 'instrument_value_id',
       'instrument_value_label','Unnamed: 0'])

feature_cols = ["line_"+str(i+1) for i in range(37)]

In [135]:
def convert(input_matrix, i):
    for col, row in input_matrix.items():
        if col in feature_cols:
            for item in conversion_guide[col]:
                if int(item["code"])==int(row[i]):
                    soc_id = input_matrix.loc[i, "society_id"]
                    code_1 = item["code_1"]
                    for item2 in conversion_guide[col]:
                        if int(item2["code_1"])==code_1 and item2["code_2"]==None and item2["code_3"]==None:
                            df.loc[i,col] = item2["display_code"]

In [136]:
for i in range(len(df)):
    convert(df[i:(i+1)],i)

In [138]:
for i, row in df.iterrows():
    culture = row['Preferred_name']
    try:
        lat = metadata[metadata['Preferred_name']==culture].reset_index()['Local_latitude'][0]
        lng = metadata[metadata['Preferred_name']==culture].reset_index()['Local_longitude'][0]
    except:
        df.drop([i])
    df.loc[i, ['lat']] = lat
    df.loc[i, ['lng']] = lng

In [141]:
df.to_csv('./output/single_modal_profile.csv')